![Image_jeu_Mastermind](https://upload.wikimedia.org/wikipedia/commons/2/2d/Mastermind.jpg)

# Règle du jeu

Le jeu se joue à deux : un codificateur et un décodeur.
Le but est de deviner, par déductions successives, la couleur de chacun des 4 pions cachés derrière un écran.
Les couleurs possibles sont au nombre de 6 (rouge, vert, bleu, jaune, orange, violet).

Déroulement du jeu : le codificateur pose à l'endroit prévu à cet effet, les 4 pions de son choix. Il doit prendre soin de ne pas révéler la couleur et la répartition dans les trous des pions. Rien ne l'empêche d'en choisir plusieurs d'une même couleur.

Son adversaire, le décodeur, est chargé de déchiffrer ce code secret. Il doit le faire en 10 coups maximum. Il place 4 ou 5 pions dans les trous de la première rangée immédiatement près de lui. Si l'un des pions correspond par sa position et sa couleur à un pion caché derrière l'écran, le codificateur l'indique en plaçant une fiche noire dans l'un des trous de marque, sur le côté droit correspondant du plateau. Si l'un des pions correspond uniquement par sa couleur, le Codificateur l'indique par une fiche blanche dans l'un des trous de marque. S'il n'y a aucune correspondance, il ne marque rien.

Le décodeur continue de poser des pions par rangées successives. La tactique consiste à sélectionner en fonction des coups précédents, couleurs et positions, de manière à obtenir le maximum d'informations de la réponse du partenaire puisque le nombre de propositions est limité par le nombre de rangées de trous du jeu. Dans la plupart des cas, il s'efforce de se rapprocher le plus possible de la solution, compte tenu des réponses précédentes, mais il peut aussi former une combinaison dans le seul but de vérifier une partie des conclusions des coups précédents et de faire en conséquence la proposition la plus propice à la déduction d'une nouvelle information.

Quand il est arrivé, au bout d'un certain nombre de coups, à placer les 4 pions qui correspondent exactement par la couleur et la position à ceux du code, la manche est terminée.

In [1]:
import random
import numpy as np
from itertools import product

TODO

In [2]:
def evaluate_guess(guess, secret_code):
    exact = sum([1 for i in range(len(secret_code)) if guess[i] == secret_code[i]])
    return exact, sum([min(guess.count(j), secret_code.count(j)) for j in set(guess)]) - exact

TODO

In [3]:
class GuessNode:
    def __init__(self, parent, parameters, guess):
        self.parameters = parameters
        self.guess = guess

        self.children = []
        self.parent = parent

        self.possible_codes = parent.possible_codes.copy()
        self.history = parent.history.copy()
        self.moves = parent.moves  # (node depth)

        self.total_moves = 0  # sum of moves of all children (for ucb1)
        self.visits = 0

    def expand(self):  # (add feedback children)

        possible_feedbacks = {}
        feedback_cache = {}

        for code in self.possible_codes:
            for guess in self.possible_codes:
                if guess not in feedback_cache:
                    feedback_cache[guess] = {}
                if code in feedback_cache[guess]:
                    feedback = feedback_cache[guess][code]
                else:
                    feedback = evaluate_guess(code, guess)
                    feedback_cache[code][guess] = feedback

                if feedback not in possible_feedbacks:
                    possible_feedbacks[feedback] = [code]
                else:
                    possible_feedbacks[feedback].append(code)

        n = len(self.possible_codes)

        for feedback in possible_feedbacks:
            child = FeedbackNode(self.parameters, feedback=feedback, parent=self,
                                 frequency=len(possible_feedbacks[feedback]) / n)
            self.children.append(child)

    def update(self, n_moves):
        self.visits += 1
        self.total_moves += n_moves

    def score(self):  # (ucb1)
        if self.visits == 0:
            return float('inf')
        return -self.total_moves / self.visits + np.sqrt(2 * np.log(self.parent.visits) / self.visits)

    def simulate(self):
        moves, possible_codes = self.moves, self.possible_codes.copy()
        secret_code = random.choice(self.possible_codes)
        while True:
            guess = random.choice(possible_codes)
            feedback = evaluate_guess(guess, secret_code)
            if feedback == (self.parameters["code_length"], 0):
                return moves
            new_possible_codes = []
            for code in possible_codes:
                if evaluate_guess(guess, code) == feedback:
                    new_possible_codes.append(code)
            possible_codes = new_possible_codes

class FeedbackNode:
    def __init__(self, parameters, feedback=None, parent=None, frequency=1.):
        self.parameters = parameters
        self.feedback = feedback
        self.frequency = frequency

        if parent:
            self.moves = parent.moves
            self.guess = parent.guess
            self.history = parent.history.copy().append((parent.guess, feedback))
            self.possible_codes = [code for code in parent.possible_codes if
                                   evaluate_guess(self.guess, secret_code=code) == feedback]
        else:
            self.moves = 0
            self.guess = None
            self.history = []
            self.possible_codes = list(
                product(range(1, self.parameters["num_colors"] + 1), repeat=self.parameters["code_length"]))
        self.parent = parent
        self.children = []

        self.expand()

    def expand(self):  # (add_guess_children)
        for code in self.possible_codes:
            child = GuessNode(self, self.parameters, code)
            self.children.append(child)

    def score(self):
        return self.frequency / self.parent.visits

    def update(self, moves):
        pass


In [4]:
def select(root):
    guess_nodes = root.children
    chosen_guess_node = max(guess_nodes, key=lambda guess_node: guess_node.score())

    if chosen_guess_node.children:
        feedback_nodes = chosen_guess_node.children
        frequencies = [node.frequency for node in feedback_nodes]
        total_frequency = np.sum(frequencies)
        normalized_probabilities = [freq / total_frequency for freq in frequencies]

        chosen_feedback_node = np.random.choice(feedback_nodes, p=normalized_probabilities)

        return chosen_feedback_node

    return chosen_guess_node

    while node.children:
        node = max(node.children, key=lambda child: child.ucb1())

    return node

In [5]:
def expand(guess_node):
    guess_node.expand()


def simulate(guess_node):
    return guess_node.simulate()  # (moves)


def backpropagate(node, total_moves):
    while node is not None:
        node.update(total_moves)
        node = node.parent

In [6]:
def best_child(feedback_node):
    return max(feedback_node.children, key=lambda child: child.visits) if feedback_node.children else None

In [7]:
def mcts_training(num_iterations):
    parameters = {
        "code_length": 2,
        "num_colors": 3
    }
    root = FeedbackNode(parameters)
    for _ in range(num_iterations):
        guess_node = select(root)
        expand(guess_node)
        total_moves = simulate(guess_node)
        backpropagate(guess_node, total_moves)

In [8]:
mcts_training(10)

AttributeError: 'NoneType' object has no attribute 'copy'